In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("./web_sherlock/xg_samp_re_0.1_valid_result.csv")
df_valid = pd.read_csv("./we_data/validation.csv")

In [4]:
df.shape

(303925, 3)

In [5]:
df_valid.shape

(303925, 25)

In [6]:
df.head()

click  click_pred_per  click_pred_class
0      0        0.000199                 0
1      0        0.000127                 0
2      0        0.000127                 0
3      0        0.000140                 0
4      0        0.000111                 0

In [7]:
pCTR = df['click_pred_per'].as_matrix()

In [8]:
# parameters
# LOW = 10
# HIGH = 40
total_budget = 6250000
payprice = df_valid['payprice'].as_matrix()
Train_df = df_valid
avgCTR = 1793/2430981 #train set
#tuning...
# base_bid = 300

In [9]:
# bidprices = base_bid*test_predictions/avgCTR

In [10]:
def generate_bidprice(base_bid, test_predictions):
    avgCTR = 1793/2430981#train set
    bidprices = base_bid*test_predictions/avgCTR
    return bidprices

In [11]:
def Prob3(Train_df,payprice,base_bid,test_predictions):
    Train_Const_summ = pd.DataFrame({'Clicks':'', 'Spend':'', 'CTR':'', 'CPM':'', 'eCPC':''}, index=[0])
    
    current_budget = total_budget
    Train_df['win'] = 0 #initialize win label
    bidprices = generate_bidprice(base_bid, test_predictions)
    #start biding...
    for i in Train_df.index:
        if current_budget >= bidprices[i]:
            #generate bidprice:
            current_bidprice = bidprices[i]

            #Winning criterion:
            current_payprice = payprice[i]
            if current_bidprice > current_payprice:
                Train_df.at[i, 'win'] = int(1)
                current_budget = current_budget - current_payprice
            else:
                Train_df.at[i, 'win'] = int(0)
        else:
            print('lack of budget!',current_budget)
            break
    #biding end        
    
    Train_df_new = Train_df[Train_df['win']!=0]
    #wintimes,_ = Train_df_new.shape
    #print('wintimes:',wintimes)
    #print('current_budget',current_budget)

    Clk = Train_df_new['click'].sum()
    Imp = Train_df_new['click'].count()
    CTR = Clk/Imp
    Spd = total_budget - current_budget
    CPM = 1000*Spd/Imp
    eCPC = Spd/Clk
    
    Train_Const_summ.loc[0,'Clicks']=Clk
    Train_Const_summ.loc[0,'Spend']=Spd
    Train_Const_summ.loc[0,'CTR']=CTR
    Train_Const_summ.loc[0,'CPM']=CPM
    Train_Const_summ.loc[0,'eCPC']=eCPC

    print(Train_Const_summ)
    
        
    return Train_df,current_budget

In [12]:
def calculate_click(base_bid,Train_df,payprice,test_predictions):
    print('base_bid:',base_bid)
    result_df,current_budget = Prob3(Train_df,payprice,base_bid,test_predictions)
    print('current_budget:',current_budget)
    Train_df_new = result_df[result_df['win']!=0]
    wintimes,_ = Train_df_new.shape
    print('wintimes:',wintimes)
    print('----------------')
    Clk = Train_df_new['click'].sum()    
    return Clk

In [13]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import mpld3
import numpy as np

In [13]:
base_bids = np.arange(10,500,5)
Clks = [calculate_click(base_bid,Train_df,payprice,pCTR) for base_bid in base_bids]
print('maxClk:', max(Clks))

base_bid: 10
current_budget: 6058577
wintimes: 8169
----------------
base_bid: 15
current_budget: 5925538
wintimes: 13404
----------------
base_bid: 20
current_budget: 5767825
wintimes: 18621
----------------
base_bid: 25
current_budget: 5600388
wintimes: 24005
----------------
base_bid: 30
current_budget: 5414241
wintimes: 29586
----------------
base_bid: 35
current_budget: 5200100
wintimes: 35586
----------------
base_bid: 40
current_budget: 4968408
wintimes: 41816
----------------
base_bid: 45
current_budget: 4717627
wintimes: 48219
----------------
base_bid: 50
current_budget: 4449719
wintimes: 54828
----------------
base_bid: 55
current_budget: 4153439
wintimes: 61693
----------------
base_bid: 60
current_budget: 3860517
wintimes: 68293
----------------
base_bid: 65
current_budget: 3570253
wintimes: 74532
----------------
base_bid: 70
current_budget: 3287201
wintimes: 80518
----------------
base_bid: 75
current_budget: 3013181
wintimes: 86055
----------------
base_bid: 80
current_

lack of budget! 16767
current_budget: 16767
wintimes: 91250
----------------
base_bid: 495
lack of budget! 98424
current_budget: 98424
wintimes: 89876
----------------
maxClk: 166


In [14]:
fig = plt.figure()
plt.plot(base_bids,Clks)
mpld3.display(fig)

In [15]:
base_bids = np.arange(90,150,5)
Clks = [calculate_click(base_bid,Train_df,payprice,pCTR) for base_bid in base_bids]
print('maxClk:', max(Clks))

base_bid: 90
current_budget: 2186000
wintimes: 101578
----------------
base_bid: 95
current_budget: 1912869
wintimes: 106467
----------------
base_bid: 100
current_budget: 1641469
wintimes: 111249
----------------
base_bid: 105
current_budget: 1369430
wintimes: 115877
----------------
base_bid: 110
current_budget: 1103844
wintimes: 120266
----------------
base_bid: 115
current_budget: 842460
wintimes: 124524
----------------
base_bid: 120
current_budget: 580884
wintimes: 128648
----------------
base_bid: 125
current_budget: 322050
wintimes: 132644
----------------
base_bid: 130
current_budget: 63976
wintimes: 136505
----------------
base_bid: 135
lack of budget! 10482
current_budget: 10482
wintimes: 135951
----------------
base_bid: 140
lack of budget! 67818
current_budget: 67818
wintimes: 133050
----------------
base_bid: 145
lack of budget! 56076
current_budget: 56076
wintimes: 131591
----------------
maxClk: 166


In [16]:
fig = plt.figure()
plt.plot(base_bids,Clks)
mpld3.display(fig)

In [17]:
base_bids = np.arange(120,140,5)
Clks = [calculate_click(base_bid,Train_df,payprice,pCTR) for base_bid in base_bids]
print('maxClk:', max(Clks))

base_bid: 120
current_budget: 580884
wintimes: 128648
----------------
base_bid: 125
current_budget: 322050
wintimes: 132644
----------------
base_bid: 130
current_budget: 63976
wintimes: 136505
----------------
base_bid: 135
lack of budget! 10482
current_budget: 10482
wintimes: 135951
----------------
maxClk: 166


In [18]:
fig = plt.figure()
plt.plot(base_bids,Clks)
mpld3.display(fig)

In [19]:
base_bids = np.arange(120,135,1)
Clks = [calculate_click(base_bid,Train_df,payprice,pCTR) for base_bid in base_bids]
print('maxClk:', max(Clks))

base_bid: 120
current_budget: 580884
wintimes: 128648
----------------
base_bid: 121
current_budget: 528844
wintimes: 129446
----------------
base_bid: 122
current_budget: 477055
wintimes: 130252
----------------
base_bid: 123
current_budget: 426091
wintimes: 131020
----------------
base_bid: 124
current_budget: 373679
wintimes: 131818
----------------
base_bid: 125
current_budget: 322050
wintimes: 132644
----------------
base_bid: 126
current_budget: 268308
wintimes: 133448
----------------
base_bid: 127
current_budget: 217442
wintimes: 134216
----------------
base_bid: 128
current_budget: 169456
wintimes: 134947
----------------
base_bid: 129
current_budget: 117094
wintimes: 135726
----------------
base_bid: 130
current_budget: 63976
wintimes: 136505
----------------
base_bid: 131
current_budget: 13345
wintimes: 137251
----------------
base_bid: 132
lack of budget! 38505
current_budget: 38505
wintimes: 136382
----------------
base_bid: 133
lack of budget! 707
current_budget: 707
wint

In [20]:
fig = plt.figure()
plt.plot(base_bids,Clks)
mpld3.display(fig)

In [21]:
base_bids = np.arange(130,135,1)
Clks = [calculate_click(base_bid,Train_df,payprice,pCTR) for base_bid in base_bids]
print('maxClk:', max(Clks))

base_bid: 130
current_budget: 63976
wintimes: 136505
----------------
base_bid: 131
current_budget: 13345
wintimes: 137251
----------------
base_bid: 132
lack of budget! 38505
current_budget: 38505
wintimes: 136382
----------------
base_bid: 133
lack of budget! 707
current_budget: 707
wintimes: 136841
----------------
base_bid: 134
lack of budget! 90
current_budget: 90
wintimes: 136522
----------------
maxClk: 167


In [22]:
fig = plt.figure()
plt.plot(base_bids,Clks)
mpld3.display(fig)

In [ ]:
#result:base_bid = 131

In [14]:
base_bid = 131
calculate_click(base_bid,Train_df,payprice,pCTR)

base_bid: 131
       CPM         CTR Clicks    Spend     eCPC
0  45439.8  0.00121675    167  6236655  37345.2
current_budget: 13345
wintimes: 137251
----------------


167

In [ ]:
#thisisend

In [ ]:
maxClks

In [ ]:
max(maxClks)

In [ ]:
# parameters
# LOW = 10
# HIGH = 40
total_budget = 6250000
payprice = df_valid['payprice'].as_matrix()
Train_df = df_valid
avgCTR = 1793/2430981 #train set
#tuning...
base_bid = 68

In [ ]:
Train_df,current_budget = Prob3(Train_df,payprice,base_bid,test_predictions)

In [ ]:
Train_Const_summ = pd.DataFrame({'Clicks':'', 'Spend':'', 'CTR':'', 'CPM':'', 'eCPC':''}, index=[0])
Train_df_new = Train_df[Train_df['win']!=0]
wintimes,_ = Train_df_new.shape
print('wintimes:',wintimes)
print('current_budget',current_budget)

Clk = Train_df_new['click'].sum()
Imp = Train_df_new['click'].count()
CTR = Clk/Imp
Spd = total_budget - current_budget
CPM = 1000*Spd/Imp
eCPC = Spd/Clk
    
Train_Const_summ.loc[0,'Clicks']=Clk
Train_Const_summ.loc[0,'Spend']=Spd
Train_Const_summ.loc[0,'CTR']=CTR
Train_Const_summ.loc[0,'CPM']=CPM
Train_Const_summ.loc[0,'eCPC']=eCPC

print(Train_Const_summ)

In [ ]:
# #字典中的key值即为csv中列名
# dataframe = pd.DataFrame({'bidprice':bidprices})

# #将DataFrame存储为csv,index表示是否显示行名，default=True
# dataframe.to_csv("bidprice10-40.csv",index=False,sep=',')

In [ ]:
# group_df = pd.read_csv('/Users/qrdai/web_project/we_data/train.csv')